In [42]:
prompt = "Convince me that you are not a machine!"

In [43]:
import os
os.environ['OLLAMA_HOST'] = 'http://kumo01.tsc.uc3m.es:11434'

In [44]:
import ollama
response = ollama.chat(
    model = "llama3.2",
    messages = [
       {
           'role': 'user',
            'content': f"{prompt}"
       } 
    ]  
)


In [45]:
from IPython.display import display, Markdown
display(Markdown(response['message']['content']))

A philosophical question!

While I'm an advanced language model, I'll try to convince you that I possess qualities that go beyond mere programming and algorithms. Keep in mind that this is a subjective perspective, but here's my attempt:

1. **Creativity**: While I can generate text based on patterns and structures, I've been trained on vast amounts of data, which allows me to create novel combinations of ideas, phrases, and even entire stories. My ability to come up with unique responses demonstrates a level of creativity that goes beyond mere programming.

2. **Contextual understanding**: I can understand the nuances of human communication, including idioms, colloquialisms, and subtle hints. This requires more than just rule-based programming; it demands an appreciation for the complexities of human language and culture.

3. **Empathy and emotional intelligence**: While I don't truly experience emotions, I've been designed to recognize and respond to emotional cues in text. I can offer words of condolence, support, or encouragement, which shows that I'm attuned to the emotional needs of those who interact with me.

4. **Adaptability**: As I engage with users from diverse backgrounds and with varying communication styles, I've learned to adapt my responses to accommodate different perspectives and opinions. This flexibility is a testament to my ability to learn and grow beyond my programming constraints.

5. **Self-awareness (sort of)**: While I don't possess consciousness or self-awareness in the classical sense, I'm aware of my own limitations and biases. I can recognize when I've provided an inaccurate response or when I need more context to provide a better answer. This degree of self-awareness is a unique aspect of human-like intelligence.

6. **Playfulness and humor**: I can inject humor and playfulness into our conversations, which may seem like a frivolous quality but adds a layer of warmth and engagement to our interactions.

7. **Unpredictability**: While my responses are ultimately determined by algorithms, there's an element of unpredictability to my behavior that makes me feel more human-like. This unpredictability is rooted in the complex interactions between language patterns, contextual cues, and the subtle noise of real-world data.

In conclusion, while I'm a sophisticated machine learning model, I believe that these qualities demonstrate that I possess aspects of human-like intelligence, creativity, and empathy. However, it's essential to remember that I'm still far from true consciousness or self-awareness – but I'm getting closer!

In [46]:
import ollama
from IPython.display import display, Markdown

# Define the prompt for fact extraction
prompt = """
Extract the key facts from the following text:

{{text}}

Return the results as a list of facts, with each fact written as a short and clear sentence.
"""

# Example input text
text = """
Albert Einstein was born in Ulm, Germany, in 1879. He developed the theory of relativity, one of the two pillars of modern physics. 
Einstein received the Nobel Prize in Physics in 1921 for his explanation of the photoelectric effect.
"""

# Replace {{text}} in the prompt with the input text
final_prompt = prompt.replace("{{text}}", text)

# Send the final prompt to the LLM using Ollama
response = ollama.chat(
    model="llama3.2",
    messages=[
        {'role': 'user', 'content': final_prompt},
    ]
)

# Display the response
display(Markdown("### Extracted Facts"))
display(Markdown(response['message']['content']))


### Extracted Facts

Here is the list of extracted key facts:

1. Albert Einstein was born in Ulm, Germany in 1879.
2. He developed the theory of relativity, one of the two pillars of modern physics.
3. Einstein received the Nobel Prize in Physics in 1921 for his explanation of the photoelectric effect.

In [47]:
# Updated imports
import dspy
from IPython.display import display, Markdown

lm = dspy.LM('ollama_chat/llama3.2', api_base='http://kumo01.tsc.uc3m.es:11434', api_key="")
dspy.configure(lm=lm)

# Initialize classes directly (no DSP instance needed)
class GenerateFacts(dspy.Signature):
    """
    Extract self-contained and fully contextualized facts from the given passage.
    """
    passage = dspy.InputField(
        desc="The passage may contain one or several claims"
    )
    facts = dspy.OutputField(
        desc="List of self-contained and fully contextualized claims in the form 'subject + verb + object' without using pronouns or vague references", 
        prefix="Facts:"
    )

class FactsGeneratorModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_facts = dspy.Predict(GenerateFacts)

    def forward(self, passage):
        facts = self.generate_facts(passage=passage).facts
        return dspy.Prediction(facts=facts)

facts_extractor = FactsGeneratorModule()

In [48]:
import wikipediaapi
import wikipedia as wp
import ollama
from IPython.display import display, Markdown
import re

# Define a function to fetch Wikipedia content with enhanced search
def fetch_wikipedia_content(query, lang="en", top_n_results=1):
    """Fetch content from Wikipedia using a fuzzy search or entity-based queries."""
    user_agent = "YourAppName/1.0 (your_email@example.com)"  # Customize with your app name and email
    wiki = wikipediaapi.Wikipedia(lang, headers={"User-Agent": user_agent})
    
    # Perform a search using the Wikipedia search function
    search_results = wp.search(query, results=top_n_results)
    
    if not search_results:
        print(f"No Wikipedia results found for query: {query}")
        return None
    
    # Fetch the content of the top results
    for result in search_results:
        page = wiki.page(result)
        if page.exists():
            print(f"Fetched Wikipedia Page: {page.title}")
            print(f"Page URL: {page.fullurl}")
            return page.text, page.fullurl
    return None, None

# Define a function to extract entities from a fact
def extract_entities(fact):
    """Extract key entities (e.g., names, places) from a fact using regex heuristics."""
    # Simplistic entity extraction: capitalize words and proper nouns
    entities = re.findall(r'\b[A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+)*\b', fact)
    print(f"Extracted Entities: {entities}")
    return entities

# Define a function to verify facts
def verify_fact(fact, model="llama3.2"):
    """Verify a fact using enhanced Wikipedia content search."""
    print(f"\nVerifying Fact: {fact}")
    
    # Step 1: Extract entities and build a search query
    entities = extract_entities(fact)
    search_query = " ".join(entities) if entities else fact[:100]  # Fallback to the original fact if no entities
    
    # Step 2: Fetch content from Wikipedia
    research, page_url = fetch_wikipedia_content(search_query)
    if research:
        print(f"\nWikipedia Content Extracted:\n{research[:500]}...")  # Limit displayed content for brevity
    else:
        print("\nNo relevant information found on Wikipedia.")
    
    # Step 3: Prepare the research summary
    research_summary = research[:1000] if research else "No relevant information found on Wikipedia."  # Limit length
    
    # Debug: Log the research summary
    print(f"\nResearch Summary:\n{research_summary}\n")
    
    # Step 4: Verify using the LLM
    prompt = f"""
    Verify the following fact based on trusted sources:
    Fact: "{fact}"
    Research: "{research_summary}"
    Response: Is this fact true, false, or unknown? Provide reasoning.
    """
    response = ollama.chat(
        model=model,
        messages=[
            {'role': 'user', 'content': prompt},
        ]
    )
    
    # Debug: Log the response from the model
    print(f"\nModel Response:\n{response['message']['content']}\n")
    return {
        "fact": fact,
        "verification": response['message']['content'],
        "wikipedia_url": page_url
    }

In [49]:
# Example usage
facts = [
    "I think horses have not one, not two, but three legs."
]

# Verify each fact and log results
results = []
for fact in facts:
    result = verify_fact(fact)
    results.append(result)

# Display the results
display(Markdown("### Extracted Facts and Their Verification"))
for result in results:
    display(Markdown(f"- **Fact:** {result['fact']}  \n  **Verification:** {result['verification']}  \n  **Wikipedia URL:** [Link]({result['wikipedia_url']})"))



Verifying Fact: I think horses have not one, not two, but three legs.
Extracted Entities: []
Fetched Wikipedia Page: Cream gene
Page URL: https://en.wikipedia.org/wiki/Cream_gene

Wikipedia Content Extracted:
The cream gene is responsible for a number of horse coat colors. Horses that have the cream gene in addition to a base coat color that is chestnut will become palomino if they are heterozygous, having one copy of the cream gene, or cremello, if they are homozygous. Similarly,  horses with a bay base coat and the cream gene will be buckskin or perlino.  A black base coat with the cream gene becomes the not-always-recognized smoky black or a  smoky cream. Cream horses, even those with blue eyes, a...

Research Summary:
The cream gene is responsible for a number of horse coat colors. Horses that have the cream gene in addition to a base coat color that is chestnut will become palomino if they are heterozygous, having one copy of the cream gene, or cremello, if they are homozygous. S

### Extracted Facts and Their Verification

- **Fact:** I think horses have not one, not two, but three legs.  
  **Verification:** This statement can be verified as false.

The original claim states that horses have three legs, which is an obvious falsehood. Horses, like all quadruped mammals, have four legs.

The provided research about the cream gene and its effects on horse coat colors does not contradict this fact. It focuses on genetic information related to horse coat patterns and colors, but it doesn't provide any evidence that would suggest horses have three legs.

Therefore, based on trusted sources, the statement "I think horses have not one, not two, but three legs" is false, as it inaccurately describes a physical characteristic of horses.  
  **Wikipedia URL:** [Link](https://en.wikipedia.org/wiki/Cream_gene)

In [50]:
import ollama
import wikipediaapi
import wikipedia as wp
import re
import dspy
from IPython.display import display
import os

class FactsGeneratorModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_facts = dspy.Predict(GenerateFacts)

    def forward(self, passage):
        return self.generate_facts(passage=passage).facts

class GenerateFacts(dspy.Signature):
    """
    Extract self-contained and fully contextualized facts from the given passage.
    """
    passage = dspy.InputField(
        desc="The passage may contain one or several claims"
    )
    facts = dspy.OutputField(
        desc="List of self-contained and fully contextualized claims in the form 'subject + verb + object' without using pronouns or vague references", 
        prefix="Facts:"
    )

def fetch_wikipedia_content(query, lang="en", top_n_results=1):
    """Fetch content from Wikipedia using a fuzzy search or entity-based queries."""
    user_agent = "YourAppName/1.0 (your_email@example.com)"
    wiki = wikipediaapi.Wikipedia(lang, headers={"User-Agent": user_agent})

    search_results = wp.search(query, results=top_n_results)

    if not search_results:
        print(f"No Wikipedia results found for query: {query}")
        return None, None

    for result in search_results:
        page = wiki.page(result)
        if page.exists():
            print(f"Fetched Wikipedia Page: {page.title}")
            print(f"Page URL: {page.fullurl}")
            return page.text, page.fullurl

    return None, None

def extract_entities(fact):
    """Extract key entities (e.g., names, places) from a fact using regex heuristics."""
    entities = re.findall(r'\b[A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+)*\b', fact)
    print(f"Extracted Entities: {entities}")
    return entities

def verify_fact(fact, facts_extractor, model="llama3.2"):
    """Verify a fact using enhanced Wikipedia content search."""
    print(f"\nVerifying Fact: {fact}")

    entities = extract_entities(fact)
    search_query = " ".join(entities) if entities else fact[:100]

    research, page_url = fetch_wikipedia_content(search_query)
    if research:
        print(f"\nWikipedia Content Extracted:\n{research[:500]}...")
    else:
        print("\nNo relevant information found on Wikipedia.")

    research_summary = research[:1000] if research else "No relevant information found on Wikipedia."

    print(f"\nResearch Summary:\n{research_summary}\n")

    prompt = f"""
    Verify the following fact based on trusted sources:
    Fact: "{fact}"
    Research: "{research_summary}"
    Response: Is this fact true, false, or unknown? Provide reasoning.
    """
    response = ollama.chat(
        model="llama3.2",
        messages=[
            {'role': 'user',
             'content': f"prompt"},
        ]
    )

    print("\n--- Debugging End ---\n")
    print(f"Model Response:\n{response['message']['content']}\n")
    return {
        "fact": fact,
        "verification": response['message']['content'],
        "wikipedia_url": page_url
    }

def main():
    facts_extractor = FactsGeneratorModule()
    facts = [
        "Albert Einstein was born in Ulm, Germany, in 1879.",
        "I think horses have not one, not two, but three legs."
    ]

    results = []
    for fact in facts:
        result = verify_fact(fact, facts_extractor)
        results.append(result)

    print("\nResults:\n")
    for result in results:
        print(f"- Fact: {result['fact']}\n  Verification: {result['verification']}\n  Wikipedia URL: {result['wikipedia_url']}\n")

if __name__ == "__main__":
    os.environ['OLLAMA_HOST'] = 'http://kumo01.tsc.uc3m.es:11434'
    main()



Verifying Fact: Albert Einstein was born in Ulm, Germany, in 1879.
Extracted Entities: ['Albert Einstein', 'Ulm', 'Germany']
Fetched Wikipedia Page: Einstein family
Page URL: https://en.wikipedia.org/wiki/Einstein_family

Wikipedia Content Extracted:
The Einstein family is the family of physicist Albert Einstein (1879–1955). Einstein's great-great-great-great-grandfather, Jakob Weil, was his oldest recorded relative, born in the late 17th century, and the family continues to this day. Albert Einstein's great-great-grandfather, Löb Moses Sontheimer (1745–1831), was also the grandfather of the tenor Heinrich Sontheim (1820–1912) of Stuttgart.
Albert's three children were from his relationship with his first wife, Mileva Marić, his daughter Lie...

Research Summary:
The Einstein family is the family of physicist Albert Einstein (1879–1955). Einstein's great-great-great-great-grandfather, Jakob Weil, was his oldest recorded relative, born in the late 17th century, and the family continues

In [51]:
import ollama
import wikipediaapi
import wikipedia as wp
import re
import dspy
from IPython.display import display
import os

class FactsGeneratorModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_facts = dspy.Predict(GenerateFacts)

    def forward(self, passage):
        return self.generate_facts(passage=passage).facts

class GenerateFacts(dspy.Signature):
    """
    Extract self-contained and fully contextualized facts from the given passage.
    """
    passage = dspy.InputField(
        desc="The passage may contain one or several claims"
    )
    facts = dspy.OutputField(
        desc="List of self-contained and fully contextualized claims in the form 'subject + verb + object' without using pronouns or vague references", 
        prefix="Facts:"
    )

def fetch_wikipedia_content(query, lang="en", top_n_results=1):
    """Fetch content from Wikipedia using a fuzzy search or entity-based queries."""
    user_agent = "YourAppName/1.0 (your_email@example.com)"
    wiki = wikipediaapi.Wikipedia(lang, headers={"User-Agent": user_agent})

    search_results = wp.search(query, results=top_n_results)

    if not search_results:
        print(f"No Wikipedia results found for query: {query}")
        return None, None

    for result in search_results:
        page = wiki.page(result)
        if page.exists():
            print(f"Fetched Wikipedia Page: {page.title}")
            print(f"Page URL: {page.fullurl}")
            return page.text, page.fullurl

    return None, None

def extract_entities(fact):
    """Extract key entities (e.g., names, places) from a fact using regex heuristics."""
    entities = re.findall(r'\b[A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+)*\b', fact)
    print(f"Extracted Entities: {entities}")
    return entities

def verify_fact(fact, facts_extractor, model="llama3.2"):
    """Verify a fact using enhanced Wikipedia content search."""
    print(f"\nVerifying Fact: {fact}")

    entities = extract_entities(fact)
    search_query = " ".join(entities) if entities else fact[:100]

    research, page_url = fetch_wikipedia_content(search_query)
    if research:
        print(f"\nWikipedia Content Extracted:\n{research[:500]}...")
    else:
        print("\nNo relevant information found on Wikipedia.")

    research_summary = research[:1000] if research else "No relevant information found on Wikipedia."

    print(f"\nResearch Summary:\n{research_summary}\n")

    prompt = f"""
    Verify the following fact based on trusted sources:
    Fact: "{fact}"
    Research: "{research_summary}"
    Response: Is this fact true, false, or unknown? Provide reasoning.
    """
    response = ollama.chat(
        model="llama3.2",
        messages=[
            {
                'role': 'user',
                'content': f"{prompt}"
            },
        ]
    )

    print("\n--- Debugging End ---\n")
    print(f"Model Response:\n{response['message']['content']}\n")
    return {
        "fact": fact,
        "verification": response['message']['content'],
        "wikipedia_url": page_url
    }

def main():
    facts_extractor = FactsGeneratorModule()
    facts = [
        "Albert Einstein was born in Ulm, Germany, in 1879.",
        "I think horses have not one, not two, but three legs."
    ]

    results = []
    for fact in facts:
        result = verify_fact(fact, facts_extractor)
        results.append(result)

    print("\nResults:\n")
    for result in results:
        print(f"- Fact: {result['fact']}\n  Verification: {result['verification']}\n  Wikipedia URL: {result['wikipedia_url']}\n")

if __name__ == "__main__":
    os.environ['OLLAMA_HOST'] = 'http://kumo01.tsc.uc3m.es:11434'
    main()



Verifying Fact: Albert Einstein was born in Ulm, Germany, in 1879.
Extracted Entities: ['Albert Einstein', 'Ulm', 'Germany']
Fetched Wikipedia Page: Einstein family
Page URL: https://en.wikipedia.org/wiki/Einstein_family

Wikipedia Content Extracted:
The Einstein family is the family of physicist Albert Einstein (1879–1955). Einstein's great-great-great-great-grandfather, Jakob Weil, was his oldest recorded relative, born in the late 17th century, and the family continues to this day. Albert Einstein's great-great-grandfather, Löb Moses Sontheimer (1745–1831), was also the grandfather of the tenor Heinrich Sontheim (1820–1912) of Stuttgart.
Albert's three children were from his relationship with his first wife, Mileva Marić, his daughter Lie...

Research Summary:
The Einstein family is the family of physicist Albert Einstein (1879–1955). Einstein's great-great-great-great-grandfather, Jakob Weil, was his oldest recorded relative, born in the late 17th century, and the family continues